In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib2 as url # the lib that handles the url stuff

from requests import session,Request
from bs4 import BeautifulSoup as bs
import scrapy
from scrapy.selector import HtmlXPathSelector
from scrapy.http import Request

import requests
from lxml import html

In [33]:
url="http://www.ziyoubaba.com/2016/05/29/python_catch_js"
# url="https://app.labelinsight.com/explore/product/id/4653440"

In [34]:
import requests
#传统方式抓取页面
def get_url_code(url):
    #声明header，伪装成浏览器
    headers={"user-agent":"Mozilla/5.0 (windows NT6.1;WOW64 ; rv:42 ) Gecko/20100101 Firefox/42.0"}
    session_req=requests.session() #构造一个兼容cookie的请求
    req=session_req.get(url=url,headers=headers) #向url发送get请求，获取整个页面
    req.encoding="utf-8" #指定页面的编码
    #print(req.text) #将页面的源代码输出到屏幕
    return req.text #返回页面的源代码

In [35]:
from bs4 import BeautifulSoup #导入包
def get_text_line(html):
    soup=BeautifulSoup(html, "lxml")
    text_line=soup.find('span',id='text_line') #找到id='text_line'的span标签
    print(text_line.text) #打印出来该span标签的文本
#执行一下看看效果如何
get_text_line(get_url_code(url)) #将输出一条文本

啊喽哈！这里是静态页面哦！再试试……


In [36]:
import dryscrape
# 使用dryscrape库 动态抓取页面
def get_url_dynamic(url):
    session_req=dryscrape.Session()
    session_req.visit(url) #请求页面
    response=session_req.body() #网页的文本
    #print(response)
    return response
get_text_line(get_url_dynamic(url)) #将输出一条文本

这里是动态文本，成功了！


### EDA

In [14]:
import time
import dryscrape

#==========================================
# Setup
#==========================================

email    = 'kristina.brecko@simon.rochester.edu'
password = '9xQWPnXdEkWCDR'

# set up a web scraping session
sess = dryscrape.Session(base_url = 'https://app.labelinsight.com/login')

# there are some failing HTTP requests, so we need to enter
# a more error-resistant mode (like real browsers do)
sess.set_error_tolerant(True)

# we don't need images
sess.set_attribute('auto_load_images', False)

# if we wanted, we could also configure a proxy server to use,
# so we can for example use Fiddler to monitor the requests
# performed by this script
#sess.set_proxy('localhost', 8888)

#==========================================
# GMail send a mail to self
#==========================================

# visit homepage and log in
print("Logging in...")
sess.visit('/')

email_field    = sess.at_css('#userEmail')
password_field = sess.at_css('#userPassword')
print(email_field)
email_field.set(email)
password_field.set(password)

email_field.form().submit()

# find the COMPOSE button and click it
print("Sending a mail...")
compose = sess.at_xpath('//*[contains(text(), "COMPOSE")]')
compose.click()

# compose the mail
to      = sess.at_xpath('//*[@name="to"]', timeout=10)
subject = sess.at_xpath('//*[@name="subject"]')
body    = sess.at_xpath('//*[@name="body"]')

to.set(email)
subject.set("Note to self")
body.set("Remember to try dryscrape!")

# send the mail

# seems like we need to wait a bit before clicking...
# Blame Google for this ;)
time.sleep(3)
send = sess.at_xpath('//*[normalize-space(text()) = "Send"]')
send.click()

# open the mail
print("Reading the mail...")
mail = sess.at_xpath('//*[normalize-space(text()) = "Note to self"]',
                     timeout=10)
mail.click()

# sleep a bit to leave the mail a chance to open.
# This is ugly, it would be better to find something
# on the resulting page that we can wait for
time.sleep(3)

# save a screenshot of the web page
print("Writing screenshot to 'gmail.png'")
sess.render('gmail.png')

Logging in...
None


AttributeError: 'NoneType' object has no attribute 'set'

### login facebook

In [52]:
# set up a web scraping session
sess = dryscrape.Session(base_url = 'https://www.facebook.com')

# we don't need images
sess.set_attribute('auto_load_images', False)

# visit homepage and search for a term
sess.visit('/')
q = sess.at_xpath('//*[@name="email"]')
print(q)
q.set('baidu_xiaohao1@126.com')
q = sess.at_xpath('//*[@name="pass"]')
print(q)
q.set('123456abc7')
# login_button = sess.at_xpath('//*[@id="u_0_2"]')
# login_button = sess.at_css('#u_0_2')
# login_button.click()
q.form().submit()

# save a screenshot of the web page
sess.render('facebook.png')
print("Screenshot written to 'facebook.png'")

<Node #/html/body/div[@id='viewport']/div[@id='page']/div[@id='root']/div[1]/div[@id='u_0_0']/div[2]/div[2]/form[@id='login_form']/div[1]/div/div[@id='email_input_container']/input[@id='m_login_email']>
<Node #/html/body/div[@id='viewport']/div[@id='page']/div[@id='root']/div[1]/div[@id='u_0_0']/div[2]/div[2]/form[@id='login_form']/div[1]/div/div[2]/div/div/div[1]/div/input[@id='m_login_password']>
Screenshot written to 'facebook.png'


In [100]:
# set up a web scraping session
sess = dryscrape.Session(base_url = 'https://app.labelinsight.com/login')

# we don't need images
sess.set_attribute('auto_load_images', False)

# visit homepage and search for a term
sess.visit('/')
q = sess.at_xpath('//*[@id="userEmail"]')

print(q)
q.set('kristina.brecko@simon.rochester.edu')
q = sess.at_xpath('/html/body/div[3]/div/div/div/div/div/main/form/div[2]/input')
# print(q)
q.set('9xQWPnXdEkWCDR')
login_button = sess.at_xpath('/html/body/div[2]/div/div/div/div/main/form/div[3]')
login_button.click()

# save a screenshot of the web page
sess.render('labelinsight.png')
print("Screenshot written to 'labelinsight.png'")

None


AttributeError: 'NoneType' object has no attribute 'set'

### google search result

In [16]:
import dryscrape
import sys

if 'linux' in sys.platform:
    # start xvfb in case no X is running. Make sure xvfb 
    # is installed, otherwise this won't work!
    dryscrape.start_xvfb()

search_term = 'dryscrape'

# set up a web scraping session
sess = dryscrape.Session(base_url = 'http://google.com')

# we don't need images
sess.set_attribute('auto_load_images', False)

# visit homepage and search for a term
sess.visit('/')
q = sess.at_xpath('//*[@name="q"]')
q.set(search_term)
q.form().submit()

# extract all links
for link in sess.xpath('//a[@href]'):
    print(link['href'])

# save a screenshot of the web page
sess.render('google.png')
print("Screenshot written to 'google.png'")


https://plus.google.com/?gpsrc=ogpy0&tab=wX
https://www.google.com/webhp?tab=ww
http://www.google.com/search?hl=en&tbm=isch&source=og&tab=wi
http://maps.google.com/maps?hl=en&tab=wl
https://play.google.com/?hl=en&tab=w8
http://www.youtube.com/results?gl=US&tab=w1
http://news.google.com/nwshp?hl=en&tab=wn
https://mail.google.com/mail/?tab=wm
https://www.google.com/intl/en/options/
https://drive.google.com/?tab=wo
https://www.google.com/calendar?tab=wc
http://translate.google.com/?hl=en&tab=wT
http://www.google.com/mobile/?hl=en&tab=wD
http://www.google.com/search?hl=en&tbo=u&tbm=bks&source=og&tab=wp
http://www.google.com/search?hl=en&tbo=u&tbm=shop&source=og&tab=wf
http://www.blogger.com/?tab=wj
http://www.google.com/finance?tab=we
https://photos.google.com/?tab=wq&pageId=none
http://www.google.com/search?hl=en&tbo=u&tbm=vid&source=og&tab=wv
https://docs.google.com/document/?usp=docs_alc
https://www.google.com/intl/en/options/
https://accounts.google.com/ServiceLogin?hl=en&passive=true&

Screenshot written to 'google.png'


### using webdriver

In [55]:
from selenium import webdriver
def get_url_dynamic2(url):
    driver=webdriver.Firefox(executable_path=r'/Users/fanerror/Downloads/geckodriver')
    driver.get(url) #请求页面，会打开一个浏览器窗口
    html_text=driver.page_source
    driver.quit()
    #print html_text
    return html_text
get_text_line(get_url_dynamic2(url)) #将输出一条文本

这里是动态文本，成功了！


In [60]:
def fb_login():
    driver=webdriver.Firefox(executable_path=r'/Users/fanerror/Downloads/geckodriver')
    driver.get ('https://www.facebook.com')

    driver.find_element_by_id('email').send_keys('baidu_xiaohao1@126.com')
    driver.find_element_by_id('pass').send_keys('123456abc')
    driver.find_element_by_id('loginbutton').click()
fb_login()

In [321]:
import time
def labelinsight_login():
    global driver
    
#     driver = webdriver.PhantomJS(executable_path='/Users/fanerror/Downloads/phantomjs-2.1.1-macosx/bin/phantomjs')
    driver=webdriver.Firefox(executable_path=r'/Users/fanerror/Downloads/geckodriver')
    driver.get ('https://app.labelinsight.com/login')

    try:
#         driver.find_element_by_name('userEmail').send_keys('kristina.brecko@simon.rochester.edu')
        driver.find_element_by_xpath('//*[@id="userEmail"]').send_keys('kristina.brecko@simon.rochester.edu')
#         driver.find_element_by_css_selector('#userEmail').send_keys('kristina.brecko@simon.rochester.edu')
        driver.find_element_by_name('userPassword').send_keys('9xQWPnXdEkWCDR')
#         driver.find_element_by_id('loginbutton').click()
        
        driver.find_element_by_css_selector('.Button-dYVdnu').click()
        time.sleep(1.5)
        
        driver.find_element_by_css_selector('.sc-gqjmRU > li:nth-child(3) > a:nth-child(1)').click()
        driver.find_element_by_css_selector('li.facets__facet:nth-child(2) > a:nth-child(1)').click()
        driver.find_element_by_css_selector('.filter-selector__list--PRODUCT_TYPE > li:nth-child(1) > div:nth-child(2) > span:nth-child(1)').click()
        driver.find_element_by_css_selector('li.ng-scope:nth-child(3) > div:nth-child(2) > span:nth-child(1)').click()
#         time.sleep(3)
    except: 
        print("Error \nDriver will be closed in 7 sec")
        time.sleep(7)
        driver.quit()
        print("Driver closed")
    return(driver)
        
def load_item(item_CSS_Selector):
    '''
    input: 
        item_CSS_Selector - CSS_Selector of target item
    '''
#     driver.get('https://app.labelinsight.com/explore/product/id/4662794')
    
#     driver_copy = driver.copy()
    driver.find_element_by_css_selector(item_CSS_Selector).click()
    time.sleep(3)
#     html_text = driver.page_source
    get_title_line(driver)
#     time.sleep(3)
    driver.back()
    driver.refresh()
    return(html_text)



In [324]:
# initial webdriver
driver = labelinsight_login()

# initial item pool
num = range(1,3)
item_CSS_Selector = ['article.card:nth-child(%s) > section:nth-child(1)'%i for i in num]

for CSS_Selector in item_CSS_Selector:
    html_text = load_item(CSS_Selector)
#     print(get_title_line(html_text))
    
# driver.quit()

Error 
Driver will be closed in 7 sec
Driver closed


ConnectionRefusedError: [Errno 61] Connection refused

In [213]:
def get_title_line(html_text):
    html_tmp = html_text.split('/n')
    html_tmp = html_tmp[2].split('div')
    for line in html_tmp:
        if 'no-top-margin' in line:
            this_line = line.split('&nbsp;')
            this_line = this_line[0].split('"> ')
            return(this_line[1])

In [323]:
def get_title_line(driver):
    element_Brand = driver.find_element_by_css_selector('.no-top-margin')
    print(element_Brand.text)
    element_Brand = driver.find_element_by_css_selector('.flexgrid-fixed > div:nth-child(1) > p:nth-child(2)')
    print(element_Brand.text)
    element_Manufacturer = driver.find_element_by_css_selector('div.border-bottom:nth-child(3) > p:nth-child(2)')
    print(element_Manufacturer.text)
    element_Manufacturer = driver.find_element_by_css_selector('div.border-bottom:nth-child(2) > p:nth-child(2)')
    print(element_Manufacturer.text)
    element_Manufacturer = driver.find_element_by_css_selector('.badge')
    print(element_Manufacturer.text)
    element_Manufacturer = driver.find_element_by_css_selector('div.border-bottom:nth-child(5) > p:nth-child(2)')
    print(element_Manufacturer.text)
    element_Manufacturer = driver.find_element_by_css_selector('div.border-bottom:nth-child(6) > p:nth-child(2)')
    print(element_Manufacturer.text)

### outdated codes

In [299]:
import re
def get_title_line(html_text):
    html_tmp = re.split('<div|</div>|\n',html_text)
    # print(len(html_tmp))
    for line in html_tmp:
    #     print(line)
        if '>Brand<' in line:
            line_tmp = re.split('<p|>|</p>',line)
            ind_tmp = line_tmp.index(' class="ng-binding"')
            print(line_tmp[ind_tmp+1])
        if '>Sub Brand<' in line:
            line_tmp = re.split('<p|>|</p>',line)
            ind_tmp = line_tmp.index(' class="ng-binding"')
            print(line_tmp[ind_tmp+1])
        if '>Manufacturer<' in line:
            line_tmp = re.split('<p|>|</p>',line)
            ind_tmp = line_tmp.index(' class="ng-binding"')
            print(line_tmp[ind_tmp+1])
    #     Last Collected has some unstable bugs delete for now
    #     if '>Last Collected<' in line:
    #         line_tmp = re.split('<p|>|</p>',line)
    #         ind_tmp = line_tmp.index('<span class="badge margin-left ng-binding"')
    #         print(line_tmp[ind_tmp+1])
        if '>Collected on<' in line:
            line_tmp = re.split('<p|>|</p>',line)
            ind_tmp = line_tmp.index(' class="no-horizontal-p ng-binding"')
            print(line_tmp[ind_tmp+1])
        if '>Collected by<' in line:
            line_tmp = re.split('<p|>|</p>',line)
            ind_tmp = line_tmp.index(' class="no-horizontal-p ng-binding"')
            print(line_tmp[ind_tmp+1])
